# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [17]:
!cat publications.tsv

pub_date	title	venue	excerpt	citation	url_slug	paper_url
2021-08-04	Task offloading in Edge and Cloud Computing: A survey on mathematical, artificial intelligence and control theory solutions	Computer Networks	Next generation communication networks are expected to accommodate a high number of new and resource-voracious applications that can be offered to a large range of end users. Even though end devices are becoming more powerful, the available local resources cannot cope with the requirements of these applications. This has created a new challenge called task offloading, where computation intensive tasks need to be offloaded to more resource powerful remote devices. Naturally, the Cloud Computing is a well-tested infrastructure that can facilitate the task offloading. However, Cloud Computing as a centralized and distant infrastructure creates significant communication delays that cannot satisfy the requirements of the emerging delay-sensitive applications. To this end, the concept 

## Import pandas

We are using the very handy pandas library for dataframes.

In [18]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [19]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url
0,2021-08-04,Task offloading in Edge and Cloud Computing: A...,Computer Networks,Next generation communication networks are exp...,"Saeik, F., Avgeris, M., Spatharakis, D., Santi...",task2021,https://hal.science/hal-03243071/document
1,2018-06-18,Edge computing in IoT ecosystems for UAV-enabl...,2018 IEEE international conference on smart co...,Unmanned Aerial Vehicles (UAV) facilitate the ...,"Kalatzis, N., Avgeris, M., Dechouniotis, D., P...",edge2018,https://www.researchgate.net/profile/Marios-Av...
2,2019-02-03,Where there is fire there is smoke: A scalable...,Sensors,A Cyber-Physical Social System (CPSS) tightly ...,"Avgeris, M., Spatharakis, D., Dechouniotis, D....",smoke2019,https://www.mdpi.com/1424-8220/19/3/639/pdf
3,2019-01-27,Semantic interoperability for iot platforms in...,Sensors,One of the main obstacles towards the promotio...,"Kalatzis, N., Routis, G., Marinellis, Y., Avge...",semantic2019,https://www.mdpi.com/1424-8220/19/3/528/htm
4,2019-04-03,Adaptive resource allocation for computation o...,ACM Transactions on Internet Technology (TOIT),Although mobile devices today have powerful ha...,"Avgeris, M., Dechouniotis, D., Athanasopoulos,...",toit2019,https://pure.qub.ac.uk/files/159624593/acm_lat...
5,2015-10-01,Paper Title Number 3,Journal 1,This paper is about the number 3. The number 4...,"Your Name, You. (2015). ""Paper Title Number 3....",paper-title-number-3,http://academicpages.github.io/files/paper3.pdf


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [20]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [21]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"
    
    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [22]:
!ls ../_publications/

2009-10-01-paper-title-number-1.md 2019-02-03-smoke2019.md
2010-10-01-paper-title-number-2.md 2019-04-03-toit2019.md
2015-10-01-paper-title-number-3.md 2021-08-04-paper-title-number-1.md
2018-06-18-edge2018.md             2021-08-04-task2021.md
2019-01-27-semantic2019.md         pdf.md


In [23]:
!cat ../_publications/2009-10-01-paper-title-number-1.md

---
title: "Paper Title Number 1"
collection: publications
permalink: /publication/2009-10-01-paper-title-number-1
excerpt: 'This paper is about the number 1. The number 2 is left for future work.'
date: 2009-10-01
venue: 'Journal 1'
paperurl: 'http://academicpages.github.io/files/paper1.pdf'
citation: 'Your Name, You. (2009). &quot;Paper Title Number 1.&quot; <i>Journal 1</i>. 1(1).'
---
This paper is about the number 1. The number 2 is left for future work.

[Download paper here](http://academicpages.github.io/files/paper1.pdf)

Recommended citation: Your Name, You. (2009). "Paper Title Number 1." <i>Journal 1</i>. 1(1).